## Losses

In [ ]:
from keras import backend as K
import tensorflow as tf

In [ ]:
def dice_coef(y_true, y_pred):
    
    smooth = 1e-10
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    
    return 1-dice_coef(y_true, y_pred)

def weighted_bce(y_true, y_pred):
    
    weights = (y_true * 49) + 1.
    bce = K.binary_crossentropy(y_true, y_pred)
    weighted_bce = K.mean(bce * weights)
    
    return weighted_bce

def bce_dice(y_true, y_pred):
    
    bce = K.binary_crossentropy(y_true, y_pred)
    dice = dice_coef_loss(y_true, y_pred)
    
    return bce + dice

def weighted_bce_dice(y_true, y_pred):
    
    w_bce = weighted_bce(y_true, y_pred)
    dice = dice_coef_loss(y_true, y_pred)
    
    return w_bce + dice

def lopsided_bootstrapped_ce(y_true, y_pred):
    
    y0 = (y_true - 1)*-1
    y0_bootstrapped_bce = 0.1*K.binary_crossentropy(y_true, y_pred)+(1-0.1)*K.binary_crossentropy(tf.cast(y_pred > 0.5, dtype='float32'), y_pred)
    y0_bootstrapped_bce = y0*y0_bootstrapped_bce
    
    y1 = (y_true * 3)
    y1_ce = K.binary_crossentropy(y_true, y_pred)
    y1_ce = y1*y1_ce
    
    return y0_bootstrapped_bce + y1_ce

def lopsided_bootstrapped_ce_dice(y_true, y_pred):
    
    l_btstrp_ce = lopsided_bootstrapped_ce(y_true, y_pred)
    dice = dice_coef_loss(y_true, y_pred)
    
    return l_btstrp_ce + dice